NOTEBOOK for reproduction of figures in the publication: 
Petropoulou, Ponti, Stel, Mastichiadis, to appear in A&A 
- Data are taken from Ponti et al. (2017)
- Numerical models are computed using the radiative transfer code ATHEvA (Mastichiadis & Kirk (1995) A&A; Dimitrakoudis et al. (2012) A&A)


In [ ]:
import numpy as np
import math
import pandas as pd
import csv
import matplotlib
import matplotlib.pyplot as plt
from astropy import units as u 
from astropy import constants as const 
from scipy.interpolate import interp1d 
from scipy import integrate 
from astropy.io import ascii  
from matplotlib.gridspec import GridSpec

**Constants**

In [ ]:
G=(const.G).cgs.value     #cm^3 g^-1 s^-2 Gravitational constant  
c=(const.c).cgs.value     #cm/s speed of light 
Ro=(const.R_sun).cgs.value            #cm
Mo=(const.M_sun).cgs.value               #gr Solar mass 
yr_to_sec=(u.yr).to(u.s)               #seconds
kpc=(u.kpc).to(u.cm)              #cm
pc=(u.pc).to(u.cm)              #cm
mp=(u.M_p).to(u.g)        #gr proton mass
me=(u.M_e).to(u.g)        #gr electron mass   
kb=(const.k_B).cgs.value
h = (const.h).cgs.value 
e=(const.e.gauss).value                 #statcoulomb
sigma_T=(const.sigma_T).cgs.value               
evtoerg = (u.eV).to(u.erg)
Bcr = me**2*c**3/(e*h/(2*np.pi))

# Functions

In [ ]:
def read_sed_model(path, file, Nsnaps):
    data = ascii.read(path+file)
    xf = 10**data['col1']
    yf = 10**data['col3']
    npts = int(len(xf)/(Nsnaps+1.))
    return xf, yf, npts

def read_elec_model(path, file, R_blob, Nsnaps):
    data = ascii.read(path+file)
    xf = 10**data['col1']
    yf = 10**data['col2']
    npts = int(len(xf)/(Nsnaps+1.)) 
    norm_e = (sigma_T*R_blob)**(-1)*(4*np.pi*R_blob**3/3)

    return xf, yf*norm_e, npts

def lc_index(xph, yph, npts, NN):
    lambda_NIR = 2.21e-4 # 2.2 μm
    vNIR = c/lambda_NIR
    vNIR_1 = 5e11 # 500 GHz
    nu_x_soft = np.linspace(2,10,200)*1e3*evtoerg/h
    nu_x_hard = np.linspace(10,20,200)*1e3*evtoerg/h
    L_NIR = np.zeros(len(time))
    L_NIR_1 = np.zeros(len(time)) 
    Lx_soft = np.zeros(len(time))
    Lx_hard = np.zeros(len(time)) 

    for i in range(0, nsnaps): 
        L_NIR[i] = np.interp(vNIR, xph[npts*(i):npts*(i+1)], yph[npts*(i):npts*(i+1)])
        L_NIR_1[i] = np.interp(vNIR_1, xph[npts*(i):npts*(i+1)], yph[npts*(i):npts*(i+1)])
        Lx_soft[i] = integrate.simpson(np.interp(nu_x_soft, xph[npts*i:npts*(i+1)], yph[npts*i:npts*(i+1)]/xph[npts*i:npts*(i+1)]), nu_x_soft)  
        Lx_hard[i] = integrate.simpson(np.interp(nu_x_hard, xph[npts*i:npts*(i+1)], yph[npts*i:npts*(i+1)]/xph[npts*i:npts*(i+1)]), nu_x_hard)  

    slope_NIR = []
    slope_X = []
    slope_X_std = []

    for i in range(0, nsnaps): 
        s = 2.- np.gradient(np.log10(yph[npts*i:npts*(i+1)]), np.log10(xph[npts*i:npts*(i+1)]))  # compute local photon index
        s_NIR_int = np.interp(np.log10(vNIR),np.log10(xph[npts*i:npts*(i+1)]),s)
        mask2 = ((xph[npts*i:npts*(i+1)] >= 2e3*evtoerg/h) & (xph[npts*i:npts*(i+1)] < 1e4*evtoerg/h))
        slope_NIR.append(s_NIR_int)
        slope_X.append(np.average(s[mask2])) 
        slope_X_std.append(np.std(s[mask2]))   
    
    return L_NIR, L_NIR_1, Lx_soft, Lx_hard, slope_NIR, slope_X, slope_X_std


def blob_orbit(t, vr, r0, theta0, phi0, theta_obs, phi_obs):
    r_t = r0 + vr * t

    # Polar angle is constant
    theta = theta0

    # Integrate for phi(t)
    dphi_dt = k / r_t**2
    phi_t = phi0 + np.cumsum(dphi_dt) * (t[1] - t[0])  # Numerical integration

    # Convert spherical to Cartesian coordinates
    x = r_t * np.sin(theta) * np.cos(phi_t)
    y = r_t * np.sin(theta) * np.sin(phi_t)
    z = r_t * np.cos(theta)


    # Velocity components in spherical coordinates
    v_phi = r_t * np.sin(theta0) * dphi_dt

    # Blob Lorentz factor
    gamma = 1./np.sqrt(1-(vr**2+v_phi**2))

    # Velocity components in Cartesian coordinates
    v_x = vr * np.sin(theta0) * np.cos(phi_t) - v_phi * np.sin(phi_t)
    v_y = vr * np.sin(theta0) * np.sin(phi_t) + v_phi * np.cos(phi_t)
    v_z = vr * np.cos(theta0)

    # Convert spherical to Cartesian coordinates
    nx_obs = 1 * np.sin(theta_obs) * np.cos(phi_obs)
    ny_obs = 1 * np.sin(theta_obs) * np.sin(phi_obs)
    nz_obs = 1 * np.cos(theta_obs)

    # Dot product
    dot_product = v_x * nx_obs + v_y * ny_obs + v_z * nz_obs
    doppler = (gamma*(1-dot_product))**(-1)
    return x,y,z,nx_obs,ny_obs,nz_obs, doppler

# Load data files

In [ ]:
# Define your main path
path = '/home/maria/Dropbox/SgrA-flares/files_for_paper/'

In [ ]:
# CTAO sensitivity curve
file = path + 'data/CTAO_South_30min.csv'
dat = ascii.read(file)
x05S = dat["col1"]*1e12 # in eV 
y05S= dat["col2"]

# Quiescent SED (Yuan et al. 2003)
file = path + 'data/Yuan2003_total.csv'
sed_q = pd.read_csv(file, header = None)

# SEDs 2014 flare (Ponti et al. 2017)
file = [path+'data/ir1.txt',path+'data/ir2.txt',path+'data/ir3.txt',path+'data/ir4.txt']
d1 = pd.read_csv(file[0], header = None, sep = '\s+')
d2 = pd.read_csv(file[1], header = None, sep = '\s+')
d3 = pd.read_csv(file[2], header = None, sep = '\s+')
d4 = pd.read_csv(file[3], header = None, sep = '\s+')

# light curves of 2014 flare (Ponti et al. 2017)
file = [path+'data/sinfoni.txt', path+'data/nustar.txt', path+'data/xmm.txt', path+'data/x_lc.txt']
lc1 = ascii.read(file[0])
lc2 = ascii.read(file[1])
lc3 = ascii.read(file[2])
lc4 = ascii.read(file[3])

**Epoch definitions**

In [ ]:
T_start = [2456900.47470, 2456900.48971, 2456900.49694,
           2456900.52160,2456900.53675, 
           2456900.54398,2456900.55914] 
T_end = [2456900.48164, 2456900.49665 , 2456900.50388,
         2456900.52855,2456900.54370,
         2456900.55093,2456900.56608]
T0 = 2456900.50784

tw  = np.multiply(np.subtract(T_start,T0), 24*3600)
tend = np.multiply(np.subtract(T_end,T0), 24*3600)
tc = tw + (tend-tw)/2 

# Parameters

In [ ]:
#Source parameters
d = 8.27*kpc 
M = 4.297e6*Mo #±0.013
RS = 2*G*M/c**2 
print('log(Rs [cm]) =', np.log10(RS))
Rg = RS/2
normD = 4*np.pi*d**2

# Frequencies and energy bands
lambda_NIR = 2.21e-4 
vNIR = c/lambda_NIR
vNIR_1 = 5e11
nu_x_soft = np.linspace(2,10,200)*1e3*evtoerg/h
nu_x_hard = np.linspace(10,20,200)*1e3*evtoerg/h

# Numerical SED models

## Figure 1: Temporal evolution of electron and photon distributions, Qo=const

In [ ]:
R_blob = 1.2e13
tacc = 1.5*R_blob/c
nsnaps = 34
time = np.linspace(0, nsnaps, nsnaps)*R_blob/c

# Read files
xph, yph, npts = read_sed_model(path+'R_1e13_B_8e-1/rise-34/', 'spec_obs.dat', nsnaps)
xe, ye, npts_e = read_elec_model(path+'R_1e13_B_8e-1/rise-34/', 'fort.89', R_blob, nsnaps)

# Make plot 
fsize = 20
fig, ax = plt.subplots(1, 2, figsize=(18, 6))  
plt.rc('font', size=20)
cmap_reversed = matplotlib.cm.get_cmap('Spectral_r')

NN = nsnaps 
colors = cmap_reversed(np.linspace(0,1,NN))

slope_NIR = []
slope_X = []
slope_X_std = []

for i in range(0, NN): 
    ax[1].plot(xph[npts*i:npts*(i+1)], yph[npts*i:npts*(i+1)], color=colors[i])     
    
    if (i > 2):
        ax[0].plot(xe[npts_e*i:npts_e*(i+1)], ye[npts_e*i:npts_e*(i+1)]/xe[npts_e*i:npts_e*(i+1)], color=colors[i], lw = 2) 
    
    s = 2.- np.gradient(np.log10(yph[npts*i:npts*(i+1)]), np.log10(xph[npts*i:npts*(i+1)]))  # compute local photon index
    s_NIR_int = np.interp(np.log10(vNIR),np.log10(xph[npts*i:npts*(i+1)]),s)
    mask2 = ((xph[npts*i:npts*(i+1)] >= 2e3*evtoerg/h) & (xph[npts*i:npts*(i+1)] < 1e4*evtoerg/h))
    slope_NIR.append(s_NIR_int)
    slope_X.append(np.average(s[mask2])) 
    slope_X_std.append(np.std(s[mask2]))  
    
cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap_reversed), ax = ax[1], values = (time[0:NN]/tacc))  
cbar.set_label('$t/t_{acc}$', fontsize=18, rotation=90)

ax[1].axvline(x=vNIR, color='k') 
ax[1].axvline(x=2e3*evtoerg/h, ls = '--', color='k') 
ax[1].axvline(x=1e4*evtoerg/h, ls = '--', color='k') 

ax[1].set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
ax[1].set_ylabel(r'$\nu L_{\nu}$ (erg/s)', fontsize=fsize)
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_ylim(1e32,1e36)  
ax[1].set_xlim(1e10,1e27)  

ax[0].plot([1e3, 1e5], [3e44, 3e44*(1e5/1e3)**(0-1/0.66)], ls = '--', color='k')
ax[0].text(3e3, 3e44, '$ - t_{acc}/t_{esc}$')
ax[0].set_xlabel(r'$\gamma$', fontsize=fsize)
ax[0].set_ylabel(r'$\gamma \, N_e(\gamma)$', fontsize=fsize)
ax[0].set_xscale('log')
ax[0].set_yscale('log') 
ax[0].set_xlim(1e2,3e6) 
ax[0].set_ylim(1e38,1e45) 

fig.tight_layout()

## Preparation of figures 2+3

In [ ]:
## READ files after choosing which CASE you want to plot
R_blob = 1.2e13
tacc = 1.5*R_blob/c
tesc = R_blob/c

flag = '34' 

if flag == '17':
    file1 = path+'R_1e13_B_8e-1/rise-17/'
    file2 = path+'R_1e13_B_8e-1/decay-17-1/'
    file3 = path+'R_1e13_B_8e-1/decay-17-2/'
    nsnaps1 = 17
elif flag == '34':
    file1 = path+'R_1e13_B_8e-1/rise-34/'
    file2 = path+'R_1e13_B_8e-1/decay-34-1/'
    file3 = path+'R_1e13_B_8e-1/decay-34-2/'
    nsnaps1 = 34
else: 
    print('error')
    
xph1, yph1, npts1 = read_sed_model(file1, 'spec_obs.dat', nsnaps1) 
xph2, yph2, npts2 = read_sed_model(file2, 'spec_obs.dat', nsnaps1) 
xph3, yph3, npts3 = read_sed_model(file3, 'spec_obs.dat', nsnaps1) 

npts = npts1
nsnaps = 2*nsnaps1
time =  np.linspace(0, nsnaps, nsnaps)*R_blob/c 
 
xph_a=np.concatenate((xph1[0:npts1*nsnaps1], xph2[0:npts2*nsnaps1]))
yph_a=np.concatenate((yph1[0:npts1*nsnaps1], yph2[0:npts2*nsnaps1]))

xph_b=np.concatenate((xph1[0:npts1*nsnaps1], xph3[0:npts3*nsnaps1]))
yph_b=np.concatenate((yph1[0:npts1*nsnaps1], yph3[0:npts3*nsnaps1]))

# print(xph1.shape, xph2.shape, xph_a.shape)
# print(xph1.shape, xph3.shape, xph_b.shape)

##########################################################################3
## LIGHT CURVE & PHOTON INDEX COMPUTATION

L_NIR, L_NIR_1, Lx_soft, Lx_hard, slope_NIR, slope_X, slope_X_std = lc_index(xph_a, yph_a, npts, len(time))
L_NIR_new, L_NIR_1_new, Lx_soft_new, Lx_hard_new, slope_NIR_new, slope_X_new, slope_X_std_new = lc_index(xph_b, yph_b, npts, len(time))


## Figure 2: Rising part of light curves

In [ ]:
fsize = 20
cmap = plt.cm.Set1

fig = plt.figure ( figsize = [7, 7])
gs = GridSpec(2, 1, height_ratios=[2, 1.], hspace = 0.05)
ax = fig.add_subplot(gs[0])

ax.plot(time[0:nsnaps1]/tacc, L_NIR[0:nsnaps1]/max(L_NIR), label = 'NIR $(2.2\,  \mu m)$', color=cmap.colors[0], marker = 'o',  mec='k', alpha = 0.6)
ax.plot(time[0:nsnaps1]/tacc, Lx_soft[0:nsnaps1]/max(Lx_soft), label = 'X-rays (2-10 keV)' , color=cmap.colors[1], marker = 'o', mec='k', alpha = 0.6)
ax.set_xticklabels([])

ax.set_ylabel(r'$\int d\nu F_\nu$ (normalized to peak)', fontsize = 14)
ax.legend(fontsize=14, loc='lower right')

ax = fig.add_subplot(gs[1])
ax.axhline(y=1+tacc/tesc/2, ls = ':', lw = 2, color = 'k')

ax.plot(time[1:nsnaps1]/tacc, slope_NIR[1:nsnaps1], label = 'NIR $(2.2\,  \mu m)$', color=cmap.colors[0], marker = 'o',  mec='k', alpha = 0.6)
ax.plot(time[1:nsnaps1]/tacc, slope_X[1:nsnaps1], label = 'X-rays (2-10 keV)' , color=cmap.colors[1], marker = 'o',  mec='k', alpha = 0.6)
ax.errorbar(time[1:nsnaps1]/tacc, slope_X[1:nsnaps1], yerr = slope_X_std[1:nsnaps1], color=cmap.colors[1], marker = 'o',  mec='k',alpha = 0.6)

ax.set_xlabel('$t/t_{acc}$')
ax.set_ylabel(r'$-d \log (\nu F_{\nu}) /d \log(\nu)+2$', fontsize = 14)

plt.savefig('R1e13_B8e-1.png', bbox_inches = 'tight')
plt.show()

## Figure 3: Light curves with rise and decay

In [ ]:
fsize = 20
cmap = plt.cm.Set1

fig = plt.figure ( figsize = [9,4])
gs = GridSpec(1, 1)
ax = fig.add_subplot(gs[0])
    

ax.plot(time/tacc, L_NIR/max(L_NIR), label = 'NIR $(2.2\,  \mu m)$', color=cmap.colors[0], ls = '-', lw = 2)
ax.plot(time/tacc, Lx_soft/max(Lx_soft), label = 'X-rays (2-10 keV)' , color=cmap.colors[1], ls = '-', lw = 2)
ax.plot(time/tacc, L_NIR_new/max(L_NIR_new), color=cmap.colors[0], ls = '--', lw = 2)
ax.plot(time/tacc, Lx_soft_new/max(Lx_soft_new) , color=cmap.colors[1], ls = '--', lw = 2) 


ax.set_ylabel(r'$\int d\nu F_\nu$ (normalized to peak)', fontsize = 14)
ax.set_xlabel('$t/t_{acc}$')
ax.legend(fontsize=14, loc='center right') 
plt.savefig('lc_decay_'+flag+'_R1e13_B8e-1.png',bbox_inches = 'tight')
plt.show()

## Figure 4: light curves and spectra for Qo=variable

In [ ]:
## Variable injection rate
R_blob = 1.2e13
tcr = R_blob/c
tacc = 1.5*R_blob/c

# Read files
file =  path + 'q_var/4/input_new-4.txt' # Variable injection rate (colored noise)
sed = ascii.read(file)
Anew = sed['col2']
nsnaps = len(Anew)-1
ts = np.linspace(0,len(Anew)-1,nsnaps) 

xph, yph, npts = read_sed_model(path+'q_var/4/', 'spec_obs.dat', nsnaps) 
time = np.linspace(0, nsnaps, nsnaps)*R_blob/c 

## LIGHT CURVE & PHOTON INDEX COMPUTATION
L_NIR, L_NIR_1, Lx_soft, Lx_hard, slope_NIR, slope_X, slope_X_std = lc_index(xph, yph, npts, len(time))

In [ ]:
fsize = 20
fig, ax = plt.subplots(1, 2, figsize=(18, 6))  
cmap = plt.cm.Set1

nn = nsnaps 
nn2 = 100

snaps = [27,36, 45, 90, 135]
labels = [str(round(time[27]/3600))+' hr', str(round(time[36]/3600))+' hr',
          str(round(time[45]/3600))+' hr', str(round(time[90]/3600))+' hr',str(round(time[135]/3600))+' hr']

ax[1].plot(10**sed_q[0], 10**sed_q[1], color='grey', ls = ':', lw = 3, label= 'Yuan et al. 2003')

j = 0 
for i in snaps:   
    ax[1].plot(xph[npts*i:npts*(i+1)], yph[npts*i:npts*(i+1)], color=cmap.colors[j], lw = 2, alpha = 1, label = labels[j])
    j = j+1   

ax[1].axvline(x=vNIR, color='k') 
ax[1].axvline(x=2e3*evtoerg/h, ls = '--', color='k') 
ax[1].axvline(x=1e4*evtoerg/h, ls = '--', color='k') 

ax[1].set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
ax[1].set_ylabel(r'$\nu L_{\nu}$ (erg/s)', fontsize=fsize)
ax[1].set_xscale('log')
ax[1].set_yscale('log') 
ax[1].set_ylim(1e33,1e36)  
ax[1].set_xlim(1e10,1e26) 
ax[1].legend(fontsize=14, loc = 'upper right', ncols = 2)
 
ax[0].plot(time[1:nn]/3600, L_NIR_1[1:nn]/max(L_NIR_1), label = '500 GHz', color=cmap.colors[2]) #, marker = 'o')
ax[0].plot(time[1:nn]/3600, L_NIR[1:nn]/max(L_NIR), label = 'NIR $(2.2\,  \mu m)$', color=cmap.colors[0], lw = 2) #, marker = 'o')
ax[0].plot(time[1:nn]/3600, Lx_soft[1:nn]/max(Lx_soft), label = 'X-rays (2-10 keV)' , color=cmap.colors[1], lw = 2) #, marker = 'o')
ax[0].plot(ts[0:nn]*tcr/3600, Anew[0:nn]/max(Anew[0:nn]), color = 'grey', ls = '--', lw = 2)

ax[0].hlines(y = 1.1, xmin = 20*tcr/3600, xmax = 40*tcr/3600, color = 'grey', lw = 4) 
ax[0].hlines(y = 1.1, xmin = 70*tcr/3600, xmax = 80*tcr/3600, color = 'grey', lw = 4) 


# These are in unitless percentages of the figure size. (0,0 is bottom left)
left, bottom, width, height = [0.33, 0.6, 0.16, 0.3]
ax2 = fig.add_axes([left, bottom, width, height])
ax2.plot(time[1:nn2]/3600, L_NIR_1[1:nn2]/max(L_NIR_1), color=cmap.colors[2]) #, marker = 'o')
ax2.plot(time[1:nn2]/3600, L_NIR[1:nn2]/max(L_NIR), color=cmap.colors[0], lw = 2) #, marker = 'o')
ax2.plot(time[1:nn2]/3600, Lx_soft[1:nn2]/max(Lx_soft),  color=cmap.colors[1], lw = 2) #, marker = 'o')
ax2.plot(ts[0:nn2]*tcr/3600, Anew[0:nn2]/max(Anew[0:nn]), color = 'grey', ls = '--', lw = 2)

ax2.set_xlabel('$t \, (hr)$')
ax[0].set_xlabel('$t \, (hr)$')
ax[0].set_ylabel(r'$\int d\nu F_\nu$ (normalized to peak)')

fig.tight_layout()
plt.savefig('lc_sed_q_var-run4.png',bbox_inches = 'tight')
plt.show() 


## Figure 5: flux-flux plot, Qo=variable

In [ ]:
tmin = [20*tcr, 70*tcr]
tmax = [40*tcr, 80*tcr] 
mask1 = (0.9*tmin[0] < time) & (time < 1.5*tmax[0])
mask2 = (0.9*tmin[1] < time) & (time < 1.5*tmax[1])
t1 = time[mask1]
t2 = time[mask2]

fsize = 20
fig, ax = plt.subplots(1, 2, figsize=(18, 6))  
cmap = plt.cm.Spectral
colors = np.array([((item/max(time))) for item in time])

ax[0].plot(L_NIR_1[1:]/max(L_NIR_1[1:]),L_NIR[1:]/max(L_NIR[1:]),  label = 'NIR $(2.2\,  \mu m)$', color='grey',   marker = 'o', alpha =0.2)
ax[1].plot(L_NIR_1[1:]/max(L_NIR_1[1:]), Lx_soft[1:]/max(Lx_soft[1:]), label = 'X-rays (2-10 keV)', color='grey', marker = 'o', alpha =0.2)


ax[0].scatter(L_NIR_1[mask1]/max(L_NIR_1[1:]), L_NIR[mask1]/max(L_NIR[1:]), s= 100, marker= 'o', edgecolors='k', c = colors[mask1], cmap = cmap)
sc = ax[1].scatter(L_NIR_1[mask1]/max(L_NIR_1[1:]), Lx_soft[mask1]/max(Lx_soft[1:]), s= 100, marker = 'o', edgecolors='k', c = colors[mask1], cmap = cmap)

ax[0].set_xlabel(r'$F_{500~GHz}/\max(F_{500~GHz})$')
ax[1].set_xlabel(r'$F_{500~GHz}/\max(F_{500~GHz})$')
ax[0].set_ylabel(r'$F/\max(F)$')
ax[0].legend(fontsize=16)
ax[1].legend(fontsize=16, loc = 'upper left')
ax[0].set_ylim(1e-9,1e1)
ax[0].set_xlim(1e-3,2e0)
ax[1].set_ylim(1e-7,1e1)
ax[1].set_xlim(1e-3,2e0)
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[1].set_xscale('log')
ax[1].set_yscale('log')
fig.tight_layout()
cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax = ax, values = t1/3600,pad=0.01)  
cbar.ax.set_title('Time (hr)', fontsize = 16)

plt.savefig('flux-flux_q_var_run4.png',bbox_inches = 'tight')

plt.show()

**Animated SED**

In [ ]:
import gif
gif.options.matplotlib["dpi"] = 300
fsize = 20 

In [ ]:
@gif.frame

def helper_plot(x, y, t, npts, i):
    global df 
    
    print(i)   

    fig, ax = plt.subplots(1, 1, figsize=(10, 6)) 

    plt.rc('font', size=20)

    ax.plot(x[npts*i:npts*(i+1)], y[npts*i:npts*(i+1)], lw = 2)
    ax.plot(10**sed_q[0], 10**sed_q[1], color='grey', ls = ':', lw = 5, label= 'Yuan et al. 2003)')

    ax.axvline(x=vNIR, color='k') 
    ax.axvline(x=230*1e9, color='k', ls = ':') 

    ax.axvline(x=2e3*evtoerg/h, ls = '--', color='k') 
    ax.axvline(x=1e4*evtoerg/h, ls = '--', color='k') 
    
    ax.text(1e23, 3e35, str('{:.2f}'.format((t[i]-t[0])/3600.))+ ' (hr)', fontsize=18)
    ax.set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
    ax.set_ylabel(r'$\nu L(\nu)$ (erg/s)', fontsize=fsize)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylim(1e32,1e36)  
    ax.set_xlim(1e8,1e27)  

In [ ]:
frames = []
for i in range(0,len(time)): 
    frames.append(helper_plot(xph,yph, time, npts, i))
    
gif.save(frames, "sed-q_var4-new.gif", 
         duration=20, unit="s", 
         between="startend")    

## Figure 6: application to the 2014 flare

In [ ]:
R_blob = 9e12  
tcr = R_blob/c 

file =  path + 'flare2014/'
nsnaps1 = 17
nsnaps2 = 17
xph1, yph1, npts = read_sed_model(file, 'spec_obs_1.dat', nsnaps1) 
xph2, yph2, npts = read_sed_model(file, 'spec_obs_3.dat', nsnaps2)
nsnaps =  nsnaps1 + nsnaps2

xph=np.concatenate((xph1[0:npts*nsnaps1], xph2[0:npts*nsnaps2]))
yph=np.concatenate((yph1[0:npts*nsnaps1], yph2[0:npts*nsnaps2]))

t1 = np.linspace(0, nsnaps1, nsnaps1)
t2 = np.linspace(nsnaps1+1, nsnaps1+nsnaps2+1, nsnaps2)
time = np.concatenate((t1,t2))

# print(xph1.shape, npts*(nsnaps1), npts*(nsnaps1))
# print(xph2.shape, npts*(nsnaps2), npts*(nsnaps2))
# print(xph.shape,  npts*(nsnaps1)+npts*(nsnaps2), len(time), nsnaps)

## LIGHT CURVE & PHOTON INDEX COMPUTATION
L_NIR, L_NIR_1, Lx_soft, Lx_hard, slope_NIR, slope_X, slope_X_std = lc_index(xph, yph, npts, len(time))

In [ ]:
# Compute time average spectra based on obs. epochs

# determine peak time of model light curve 
mask = (Lx_soft == max(Lx_soft)) 
tt =  (time-min(time[mask]))*tcr - 250 # apply an arbitrary shift in time 

lc_v = np.zeros((npts, nsnaps)) 
for j in range(0, npts): # energy index
    for i in range(0, nsnaps): # time index
        lc_v[j,i] = yph[npts*i:npts*(i+1)][j]

cmap = plt.cm.tab10
colors = cmap(np.linspace(0,1,npts))

for j in range(0,npts): 
    plt.plot(tt[0:nsnaps], lc_v[j,:], marker = '.' , color = colors[j], lw =0)

lc_v_int = np.zeros((npts, 4))
for k in range(0,4):
    tint = np.linspace(tw[k], tend[k],100) 
    for j in range(0, npts):
        lc_v_int[j,k] = integrate.simpson(10**np.interp(tint, tt[0:nsnaps], np.log10(lc_v[j,])), tint)/(tend[k]-tw[k])
        
        plt.plot(tint,10**np.interp(tint, tt[0:nsnaps], np.log10(lc_v[j,])), color = colors[j])

plt.ylim(1e33,1e36)
plt.yscale('log')   

### Time-average SEDs

In [ ]:
fsize = 20
fig, ax = plt.subplots(1, 1, figsize=(10, 6))  

plt.rc('font', size=20)
cmap  = plt.cm.Greys

colors = cmap(np.linspace(0,1,nsnaps))

for i in range(0, nsnaps):    
    ax.plot(xph[npts*i:npts*(i+1)], yph[npts*i:npts*(i+1)], color=colors[i], lw = 0.5, alpha = 0.8)

cmap2 = plt.cm.Set1 
for k in range (0, 4):
    ax.plot(xph[0:npts], lc_v_int[:,k], color = cmap2.colors[k], lw = 3)

cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax = ax, values = tt/3600)  
cbar.set_label('$t-t_{pk}$ (hr)', fontsize=18, rotation=90)

ax.errorbar(d1[0], d1[1], xerr = d1[3], yerr = d1[2], ls ='none', color=cmap2.colors[0], label = 'IR1')
ax.errorbar(d2[0], d2[1], xerr = d2[3], yerr = d2[2], ls ='none', color=cmap2.colors[1], label = 'IR2')
ax.errorbar(d3[0], d3[1], xerr = d3[3], yerr = d3[2], ls ='none', color=cmap2.colors[2], label = 'IR3')
ax.errorbar(d4[0], d4[1], xerr = d4[3], yerr = d4[2], ls ='none', color=cmap2.colors[3], label = 'IR4')
    
ax.scatter(d1[0], d1[1], color=cmap2.colors[0])
ax.scatter(d2[0], d2[1], color=cmap2.colors[1])
ax.scatter(d3[0], d3[1], color=cmap2.colors[2])
ax.scatter(d4[0], d4[1], color=cmap2.colors[3])

ax.plot(10**sed_q[0], 10**sed_q[1], color='grey', ls = ':', lw = 3, label= 'Yuan et al. 2003')

ax.set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
ax.set_ylabel(r'$\nu L_{\nu}$ (erg/s)', fontsize=fsize)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylim(1e34,1e36)  
ax.set_xlim(1e12,1e20) 
ax.legend(fontsize=14, loc = 'upper left', ncols = 3)

fig.tight_layout()
plt.savefig('sed_flare2014_run13.png',bbox_inches = 'tight')

### Light curves

In [ ]:
fsize = 20
fig, ax = plt.subplots(1, 1, figsize=(10, 6)) 
cmap = plt.cm.tab10

norm_NIR = 4*np.pi*d**2*vNIR*1e-26
normL = 4*np.pi*d**2                             
vX = 1.450793545250951e+18  

plt.errorbar(tc, lc1['flux(mJy)']*norm_NIR, xerr = lc1['xerror(s)'], yerr = lc1['yerror(mJy)']*norm_NIR, ls = 'none', marker='o', color=cmap.colors[1], label = 'SINFONI (2.2 $\mu$m)', alpha=0.5)
plt.scatter(lc4["col1"], lc4["col2"]*normL*1e-23*vX, label = 'XMM (2-10 keV)', alpha=0.5)
plt.errorbar(lc4["col1"], lc4["col2"]*normL*1e-23*vX, yerr = lc4["col3"]*normL*1e-23*vX, ls = "none", alpha=0.5)

labels = ['IR1', 'IR2', 'IR3', 'IR4']
cmap2 = plt.cm.tab10 
for k in range(0, 4):
    plt.hlines(xmin = tw[k], xmax = tend[k], y = 3.5e35, ls ='-', color = cmap2.colors[k], lw = 3)
    plt.text(tw[k], 3.2e35, labels[k], fontsize=12)

tt_int = np.linspace(tt[0], tt[-1], 100)

f1 = interp1d(tt, L_NIR, kind = 'quadratic')
f2 = interp1d(tt, Lx_soft, kind = 'quadratic')

plt.plot(tt_int, f1(tt_int), color=cmap.colors[1], lw = 2)
plt.plot(tt_int, f2(tt_int), color=cmap.colors[0], lw = 2)  
 

ax.set_xlabel('$t - t_{pk}$ (s)')
ax.set_ylabel('$L$  (erg/s)')
ax.legend(fontsize=14)
plt.xlim(-6000,6000)


## Figure 7: gamma-ray spectrum with CTAO sensitivity curve

In [ ]:
fsize = 20
fig, ax = plt.subplots(1, 1, figsize=(10, 6))  

plt.rc('font', size=20)

cmap2 = plt.cm.Set1 

for k in range (0, 4):
    ax.plot(xph[0:npts], lc_v_int[:,k], color = cmap2.colors[k], lw =3)
    
ax.scatter(d1[0], d1[1], color=cmap2.colors[0]) 
ax.scatter(d2[0], d2[1], color=cmap2.colors[1]) 
ax.scatter(d3[0], d3[1], color=cmap2.colors[2]) 
ax.scatter(d4[0], d4[1], color=cmap2.colors[3]) 


ax.errorbar(d1[0], d1[1], xerr = d1[3], yerr = d1[2], ls ='none', color=cmap2.colors[0], label = 'IR1', marker = '')
ax.errorbar(d2[0], d2[1], xerr = d2[3], yerr = d2[2], ls ='none', color=cmap2.colors[1], label = 'IR2', marker = '')
ax.errorbar(d3[0], d3[1], xerr = d3[3], yerr = d3[2], ls ='none', color=cmap2.colors[2], label = 'IR3', marker = '')
ax.errorbar(d4[0], d4[1], xerr = d4[3], yerr = d4[2], ls ='none', color=cmap2.colors[3], label = 'IR4', marker = '')

ax.scatter(x05S*evtoerg/h, y05S*(4*np.pi*d**2), edgecolor='k', alpha = 0.7, label='CTAO South (30min)')

ax.hlines(xmin=0.06*1e9*evtoerg/h, xmax = 0.3*1e9*evtoerg/h, y = 1.06*1e-10*(4*np.pi*d**2), lw = 1, color='black')
ax.hlines(xmin=0.3*1e9*evtoerg/h, xmax = 3*1e9*evtoerg/h, y = 1.99*1e-10*(4*np.pi*d**2),  lw = 1, color='black')
ax.hlines(xmin=3*1e9*evtoerg/h, xmax = 10*1e9*evtoerg/h, y = 0.699*1e-10*(4*np.pi*d**2),  lw = 1, color='black')
ax.hlines(xmin=10*1e9*evtoerg/h, xmax = 500*1e9*evtoerg/h, y = 0.345*1e-10*(4*np.pi*d**2), lw = 1, color='black', label='4FGL J1745.6-2859')


ax.set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
ax.set_ylabel(r'$\nu L(\nu)$ (erg/s)', fontsize=fsize)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(1e33,1e37)  
ax.set_xlim(1e22,1e29)  
ax.legend(fontsize=14, loc = 'upper right', ncols = 3)
fig.tight_layout()
plt.savefig('sed_grays_flare2014_run13.png',bbox_inches = 'tight')

## Figure A.1: effects of blob radius

In [ ]:
R_blob = [1.2e13, 1.2e12, 1.2e11]
# tacc = 1.5*R_blob/c
nsnaps_arr = [17, 17, 17] 

time1 = np.linspace(0, nsnaps_arr[0], nsnaps_arr[0])*R_blob[0]/c
time2 = np.linspace(0, nsnaps_arr[1], nsnaps_arr[1])*R_blob[1]/c
time3 = np.linspace(0, nsnaps_arr[2], nsnaps_arr[2])*R_blob[2]/c

# Read files
xph1, yph1, npts1 = read_sed_model(path+'R_1e13_B_8e-1/rise-17/', 'spec_obs.dat', nsnaps_arr[0])
xe1, ye1, npts1_e = read_elec_model(path+'R_1e13_B_8e-1/rise-17/', 'fort.89', R_blob[0], nsnaps_arr[0])

xph2, yph2, npts2 = read_sed_model(path+'R_1e12_B_3.7e0/rise-1/', 'spec_obs.dat', nsnaps_arr[1])
xe2, ye2, npts2_e = read_elec_model(path+'R_1e12_B_3.7e0/rise-1/', 'fort.89', R_blob[1], nsnaps_arr[1])

xph3, yph3, npts3 = read_sed_model(path+'R_1e11_B_1.7e1/rise-1/', 'spec_obs.dat', nsnaps_arr[2])
xe3, ye3, npts3_e = read_elec_model(path+'R_1e11_B_1.7e1/rise-1/', 'fort.89', R_blob[2], nsnaps_arr[2])


xph4, yph4, npts4 = read_sed_model(path+'R_1e11_B_1.7e1/rise-2/', 'spec_obs.dat', nsnaps_arr[2])
xe4, ye4, npts4_e = read_elec_model(path+'R_1e11_B_1.7e1/rise-2/', 'fort.89', R_blob[2], nsnaps_arr[2])


xph5, yph5, npts5 = read_sed_model(path+'R_1e11_B_1.7e1/rise-3/', 'spec_obs.dat', nsnaps_arr[2])
xe5, ye5, npts5_e = read_elec_model(path+'R_1e11_B_1.7e1/rise-3/', 'fort.89', R_blob[2], nsnaps_arr[2])


# Make plot
fsize = 20
fig, ax = plt.subplots(2, 1, figsize=(9,10))  

plt.rc('font', size=20) 
cmap = plt.cm.Set1
lwidth = 3

i = nsnaps_arr[0]-1
ax[1].plot(xph1[npts1*i:npts1*(i+1)], yph1[npts1*i:npts1*(i+1)], color=cmap.colors[0],lw = lwidth)
ax[0].plot(xe1[npts1_e*i:npts1_e*(i+1)], ye1[npts1_e*i:npts1_e*(i+1)]/xe1[npts1_e*i:npts1_e*(i+1)], 
                 label = '$R=10 \, R_S, B = 0.8 \, G$', color=cmap.colors[0],lw = lwidth)

i = nsnaps_arr[1]-1
ax[1].plot(xph2[npts2*i:npts2*(i+1)], yph2[npts2*i:npts2*(i+1)], color=cmap.colors[1],lw = lwidth)
ax[0].plot(xe2[npts2_e*i:npts2_e*(i+1)], ye2[npts2_e*i:npts2_e*(i+1)]/xe2[npts2_e*i:npts2_e*(i+1)],   
                label = '$R=R_S, B=3.7\, G$', color=cmap.colors[1],lw = lwidth)
    
i = nsnaps_arr[2]-1
ax[1].plot(xph3[npts3*i:npts3*(i+1)], yph3[npts3*i:npts3*(i+1)], color=cmap.colors[2],lw = lwidth) 
ax[0].plot(xe3[npts3_e*i:npts3_e*(i+1)], ye3[npts3_e*i:npts3_e*(i+1)]/xe3[npts3_e*i:npts3_e*(i+1)], 
                label = '$R=0.1 \, R_S, B = 17 \, G$',color=cmap.colors[2],lw = lwidth) 

i = nsnaps_arr[2]-1
ax[1].plot(xph4[npts4*i:npts4*(i+1)], yph4[npts4*i:npts4*(i+1)],  ls ='--', color=cmap.colors[2],lw = lwidth) 
ax[0].plot(xe4[npts4_e*i:npts4_e*(i+1)], ye4[npts4_e*i:npts4_e*(i+1)]/xe4[npts4_e*i:npts4_e*(i+1)], 
                 ls = '--',color=cmap.colors[2],lw = lwidth) 
    
i = nsnaps_arr[2]-1
ax[1].plot(xph4[npts4*i:npts4*(i+1)], yph4[npts4*i:npts4*(i+1)],  ls ='--', color=cmap.colors[2],lw = lwidth)     
ax[0].plot(xe4[npts4_e*i:npts4_e*(i+1)], ye4[npts4_e*i:npts4_e*(i+1)]/xe4[npts4_e*i:npts4_e*(i+1)], 
                 ls = '--',color=cmap.colors[2],lw = lwidth) 
i = nsnaps_arr[2]-1
ax[1].plot(xph5[npts5*i:npts5*(i+1)], yph5[npts5*i:npts5*(i+1)],  ls =':', color=cmap.colors[2],lw = lwidth)     
ax[0].plot(xe5[npts5_e*i:npts5_e*(i+1)], ye5[npts5_e*i:npts5_e*(i+1)]/xe5[npts5_e*i:npts5_e*(i+1)], 
                 ls = ':',color=cmap.colors[2],lw = lwidth) 
    
ax[1].axvline(x=vNIR, color='k') 

ax[1].axvline(x=2e3*evtoerg/h, ls = '--', color='k') 
ax[1].axvline(x=1e4*evtoerg/h, ls = '--', color='k') 

ax[1].set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
ax[1].set_ylabel(r'$\nu L_{\nu}$ (erg/s)', fontsize=fsize)
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_ylim(1e32,1e37)  
# ax[1].set_xlim(1e10,1e27)  
ax[1].set_xlim(1e10,1e27)

ax[0].plot([1e3, 1e5], [3e44, 3e44*(1e5/1e3)**(0-1/0.66)], ls = '--', color='k')
ax[0].text(2e3, 1e44, '$ - t_{acc}/t_{esc}$')
ax[0].set_xlabel(r'$\gamma$', fontsize=fsize)
ax[0].set_ylabel(r'$\gamma \, N_e(\gamma)$', fontsize=fsize)
ax[0].set_xscale('log')
ax[0].set_yscale('log') 

ax[0].set_xlim(1e2,1e7) 
ax[0].set_ylim(1e37,1e46) 

ax[0].legend(fontsize = 14)
fig.tight_layout()
plt.savefig('sed_edf_R.png', bbox_inches = 'tight')

## Figure A.2: effects of γ0

In [ ]:
R_blob = 1.2e13
tacc = 1.5*R_blob/c
nsnaps_arr = [17, 22, 26] 

time1 = np.linspace(0, nsnaps_arr[0], nsnaps_arr[0])*R_blob/c
time2 = np.linspace(0, nsnaps_arr[1], nsnaps_arr[1])*R_blob/c
time3 = np.linspace(0, nsnaps_arr[2], nsnaps_arr[2])*R_blob/c

# Read files
xph1, yph1, npts1 = read_sed_model(path+'R_1e13_B_8e-1/rise-17/', 'spec_obs.dat', nsnaps_arr[0])
xe1, ye1, npts1_e = read_elec_model(path+'R_1e13_B_8e-1/rise-17/', 'fort.89', R_blob, nsnaps_arr[0])

xph2, yph2, npts2 = read_sed_model(path+'R_1e13_B_8e-1/rise-22-g0_1.7/', 'spec_obs.dat', nsnaps_arr[1])
xe2, ye2, npts2_e = read_elec_model(path+'R_1e13_B_8e-1/rise-22-g0_1.7/', 'fort.89', R_blob, nsnaps_arr[1])

xph3, yph3, npts3 = read_sed_model(path+'R_1e13_B_8e-1/rise-26-g0_0.7/', 'spec_obs.dat', nsnaps_arr[2])
xe3, ye3, npts3_e = read_elec_model(path+'R_1e13_B_8e-1/rise-26-g0_0.7/', 'fort.89', R_blob, nsnaps_arr[2])

# Make plot
fsize = 20
fig, ax = plt.subplots(2, 1, figsize=(9,10))  

plt.rc('font', size=20) 
cmap = plt.cm.Set1
lwidth = 3

i = nsnaps_arr[0]-1
ax[1].plot(xph1[npts1*i:npts1*(i+1)], yph1[npts1*i:npts1*(i+1)], color=cmap.colors[0],lw = lwidth)     

if (i > 2):
    ax[0].plot(xe1[npts1_e*i:npts1_e*(i+1)], ye1[npts1_e*i:npts1_e*(i+1)]/xe1[npts1_e*i:npts1_e*(i+1)], 
                 label= '$\gamma_0=10^{2.7}, l_e = 10^{-3.5}$',color=cmap.colors[0],lw = lwidth)

i = nsnaps_arr[1]-1
ax[1].plot(xph2[npts2*i:npts2*(i+1)], yph2[npts2*i:npts2*(i+1)], color=cmap.colors[1],lw = lwidth)     

if (i > 2):
    ax[0].plot(xe2[npts2_e*i:npts2_e*(i+1)], ye2[npts2_e*i:npts2_e*(i+1)]/xe2[npts2_e*i:npts2_e*(i+1)],   
               label= '$\gamma_0=10^{1.7}, l_e = 10^{-3}$', color=cmap.colors[1],lw = lwidth)
    
i = nsnaps_arr[2]-1
ax[1].plot(xph3[npts3*i:npts3*(i+1)], yph3[npts3*i:npts3*(i+1)], color=cmap.colors[2],lw = lwidth)     
     

if (i > 2):
    ax[0].plot(xe3[npts3_e*i:npts3_e*(i+1)], ye3[npts3_e*i:npts3_e*(i+1)]/xe3[npts3_e*i:npts3_e*(i+1)], 
                label= '$\gamma_0=10^{0.7}, l_e = 10^{-2.5}$',color=cmap.colors[2],lw = lwidth) 

ax[1].axvline(x=vNIR, color='k') 
ax[1].axvline(x=2e3*evtoerg/h, ls = '--', color='k') 
ax[1].axvline(x=1e4*evtoerg/h, ls = '--', color='k') 

ax[1].set_xlabel(r'$\nu$ (Hz)', fontsize=fsize)
ax[1].set_ylabel(r'$\nu L_{\nu}$ (erg/s)', fontsize=fsize)
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_ylim(1e32,1e37)  
ax[1].set_xlim(1e10,1e27)

ax[0].plot([1e3, 1e5], [3e44, 3e44*(1e5/1e3)**(0-1/0.66)], ls = '--', color='k')
ax[0].text(2e3, 1e44, '$ - t_{acc}/t_{esc}$')
ax[0].set_xlabel(r'$\gamma$', fontsize=fsize)
ax[0].set_ylabel(r'$\gamma \, N_e(\gamma)$', fontsize=fsize)
ax[0].set_xscale('log')
ax[0].set_yscale('log') 

ax[0].set_xlim(1e0,1e7)
ax[0].set_ylim(1e38,1e48) 

ax[0].legend(fontsize = 14)
fig.tight_layout()
plt.savefig('sed_edf_g0.png', bbox_inches = 'tight')

# Effects of Doppler boosting

## Figure A.4

In [ ]:
theta_obs = 168*np.pi/180
phi_obs = np.pi/2

# Parameters
r0 = 36.0    # Initial radial distance in units of Rg
vr = 0.01  # Radial velocity in units of c
theta0 = 15*np.pi/180 #np.pi / 4  # Constant polar angle
phi0 =  200*np.pi/180 #0.0  # Initial azimuthal angle
vphi0 = 0.41 # np.sqrt(1/r0) # 0.41  # Initial azimuthal velocity in units of c 
dot_phi0 = vphi0/(r0*np.sin(theta0)) # np.sqrt(1/r0**3) # Initial azimuthal velocity 
print('dot(phi) =', dot_phi0, dot_phi0*r0*np.sin(theta0), np.sqrt(1/r0))
print('vr =', vr)
k = dot_phi0*r0**2     # Constant for dφ/dt * r(t)^2

tau = np.linspace(0, 1300, 1000)
x,y,z,nx_obs,ny_obs,nz_obs, doppler = blob_orbit(tau, vr, r0, theta0, phi0, theta_obs, phi_obs)
 
# Plot the trajectory in 3D
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, label='3D Trajectory')
ax.scatter(x[0], y[0], z[0], color='blue', marker = 'o', s = 100, label='Start Point')  # Mark the starting point
ax.scatter(x[-1], y[-1], z[-1], color='blue', marker = '*', s = 100, label='End Point')  # Mark the end point
ax.scatter(0,0,0, color='k', s = 100)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend(fontsize = 16)
fig.tight_layout()
plt.savefig('3d-trajectory.png',bbox_inches = 'tight')
plt.show()

fig = plt.figure(figsize=(10, 8))
plt.plot(tau, doppler) 
plt.ylabel('Doppler factor')
plt.xlabel('time (sec)')
plt.show()


## Figures A.5-A.10

In [ ]:
## READ files after choosing which CASE you want to plot
R_blob = 1.2e13
tacc = 1.5*R_blob/c
tesc = R_blob/c

flag = '34' 

if flag == '17':
    file1 = path+'R_1e13_B_8e-1/rise-17/'
    file2 = path+'R_1e13_B_8e-1/decay-17-1/'
    file3 = path+'R_1e13_B_8e-1/decay-17-2/'
    nsnaps1 = 17
elif flag == '34':
    file1 = path+'R_1e13_B_8e-1/rise-34/'
    file2 = path+'R_1e13_B_8e-1/decay-34-1/'
    file3 = path+'R_1e13_B_8e-1/decay-34-2/'
    nsnaps1 = 34
else: 
    print('error')
    
xph1, yph1, npts1 = read_sed_model(file1, 'spec_obs.dat', nsnaps1) 
xph2, yph2, npts2 = read_sed_model(file2, 'spec_obs.dat', nsnaps1) 
xph3, yph3, npts3 = read_sed_model(file3, 'spec_obs.dat', nsnaps1) 

npts = npts1
nsnaps = 2*nsnaps1

xph_a=np.concatenate((xph1[0:npts1*nsnaps1], xph2[0:npts2*nsnaps1]))
yph_a=np.concatenate((yph1[0:npts1*nsnaps1], yph2[0:npts2*nsnaps1]))

xph_b=np.concatenate((xph1[0:npts1*nsnaps1], xph3[0:npts3*nsnaps1]))
yph_b=np.concatenate((yph1[0:npts1*nsnaps1], yph3[0:npts3*nsnaps1]))

time =  np.linspace(0, nsnaps, nsnaps)*R_blob/c 
tt2 = time/(Rg/c) # in units of Rg/c
dt = tt2[1:] - tt2[:-1] # time diff between SED snapshots in Rg/c

##########################################################################3

## LIGHT CURVE & PHOTON INDEX COMPUTATION (without beaming)
L_NIR_noboost, L_NIR_1_noboost, Lx_soft_noboost, Lx_hard_noboost, slope_NIR, slope_X, slope_X_std = lc_index(xph_a, yph_a, npts, len(time))
L_NIR_new_noboost, L_NIR_1_new_noboost, Lx_soft_new_noboost, Lx_hard_new_noboost, slope_NIR_new, slope_X_new, slope_X_std_new = lc_index(xph_b, yph_b, npts, len(time))
 

In [ ]:
# Observer's location
theta_obs = 168*np.pi/180
phi_obs = np.pi/2

# Parameters for blob trajectory 
r0 = 36.0    # Initial radial distance in units of Rg
vr = 0.01  # Radial velocity in units of c
theta0 = 15*np.pi/180 #np.pi / 4  # Constant polar angle
phi0 =  200*np.pi/180 #0.0  # Initial azimuthal angle
vphi0 = 0.41 # np.sqrt(1/r0) # 0.41  # Initial azimuthal velocity in units of c 
dot_phi0 = vphi0/(r0*np.sin(theta0)) # np.sqrt(1/r0**3) # Initial azimuthal velocity 
k = dot_phi0*r0**2     # Constant for dφ/dt * r(t)^2


band = ['Xray', 'NIR']
params = ['phi0', 'vphi0' , 'vr']
phi0_arr = [0, 60*np.pi/180, 120*np.pi/180, 180*np.pi/180]
vphi0_arr = [0.2,0.4,0.6,0.8]
vr_arr = [0.01,0.05,0.1,0.5]

flag = params[0]
flag2 = band[0] 

if flag == params[0]:
    labels = [r'$\phi_0=0^{\rm o}$', r'$\phi_0=60^{\rm o}$', r'$\phi_0=120^{\rm o}$', r'$\phi_0=180^{\rm o}$']
if flag == params[1]:
    labels = [r'$\upsilon_{\phi_0}=0.2$', r'$\upsilon_{\phi_0}=0.4$', r'$\upsilon_{\phi_0}=0.6$', r'$\upsilon_{\phi_0}=0.8$']    
if flag == params[2]:
    labels = [r'$\upsilon_{r}=0.01$', r'$\upsilon_{r}=0.05$', r'$\upsilon_{r}=0.1$', r'$\upsilon_{r}=0.5$']

fsize = 18
cmap = plt.cm.Set1
nrows = 2 
ncols = 2

fig = plt.figure (figsize = [18,8])
gs = GridSpec(nrows= nrows, ncols = ncols)

for i in range(nrows):
    for j in range(ncols):
        if flag == params[0]: 
            phi0 = phi0_arr[i*ncols+j]  
        if flag == params[1]:           
            vphi0 = vphi0_arr[i*ncols+j] 
        
        dot_phi0 = vphi0/(r0*np.sin(theta0)) # np.sqrt(1/r0**3) # Initial azimuthal velocity
        k = dot_phi0*r0**2     # Constant for dφ/dt * r(t)^2
        
        if flag == params[1]: 
            vr = vr_arr[i*ncols+j]
            
# Compute the Doppler factor of the blob   
        x,y,z,nx_obs,ny_obs,nz_obs, doppler = blob_orbit(tt2, vr, r0, theta0, phi0, theta_obs, phi_obs)
        print('φ0, δmax = ', phi0, max(doppler))
        t_obs = tt2*(Rg/c)/doppler
        t_obs2 = [0]
        for it in range(len(doppler)-1):
            t_obs2.append(t_obs2[it]+dt[it]*(Rg/c)/doppler[it])
        
        L_NIR = np.zeros(len(time))
        Lx_soft = np.zeros(len(time))
        ## DIFFERENT DECAY
        L_NIR_new = np.zeros(len(time))
        Lx_soft_new = np.zeros(len(time))
    # Compute Light Curves with Doppler boosting 
        for ii in range(0, nsnaps): 
            L_NIR[ii] = np.interp(vNIR, xph_a[npts*(ii):npts*(ii+1)]*doppler[ii], yph_a[npts*(ii):npts*(ii+1)]*doppler[ii]**4)  
            Lx_soft[ii] = integrate.simpson(np.interp(nu_x_soft, xph_a[npts*ii:npts*(ii+1)]*doppler[ii], yph_a[npts*ii:npts*(ii+1)]/xph_a[npts*ii:npts*(ii+1)]*doppler[ii]**3), nu_x_soft)  

        ## DIFFERENT DECAY    
            L_NIR_new[ii] = np.interp(vNIR, xph_b[npts*(ii):npts*(ii+1)]*doppler[ii], yph_b[npts*(ii):npts*(ii+1)]*doppler[ii]**4)
            Lx_soft_new[ii] = integrate.simpson(np.interp(nu_x_soft, xph_b[npts*ii:npts*(ii+1)]*doppler[ii], yph_b[npts*ii:npts*(ii+1)]/xph_b[npts*ii:npts*(ii+1)]*doppler[ii]**3), nu_x_soft)  
            
        ax = fig.add_subplot(gs[i,j])
        if flag2 == band[0]:
            ax.plot(t_obs2/(Rg/c), Lx_soft, label = 'X-rays (2-10 keV)' , color=cmap.colors[1], ls = '-', lw = 4)
            ax.plot(t_obs2/(Rg/c), Lx_soft_new , color=cmap.colors[1], ls = '--', lw = 4) 
            ax.plot(time/(Rg/c), Lx_soft_noboost,  color=cmap.colors[1], ls = '-', lw = 2, alpha = 0.6)
            ax.plot(time/(Rg/c), Lx_soft_new_noboost, color=cmap.colors[1], ls = '--', lw = 2, alpha = 0.6) 
        if flag2 == band[1]:
            ax.plot(t_obs2/(Rg/c), L_NIR, label = 'NIR $(2.2\,  \mu m)$', color=cmap.colors[0], ls = '-', lw = 4)
            ax.plot(t_obs2/(Rg/c), L_NIR_new, color=cmap.colors[0], ls = '--', lw = 4)
            ax.plot(time/(Rg/c), L_NIR_noboost, color=cmap.colors[0], ls = '-', lw = 2, alpha = 0.6)        
            ax.plot(time/(Rg/c), L_NIR_new_noboost, color=cmap.colors[0], ls = '--', lw = 2, alpha = 0.6)
                
        if j == 0:
            ax.set_ylabel(r'$\int d\nu L_\nu$ (erg/s)')  
         
        ax.text(0.7,0.7, labels[i*nrows+j], transform=ax.transAxes)
        ax.set_xlabel('$t_{obs} \, (R_g/c)$') 
fig.subplots_adjust(wspace=0.1, hspace=0.5) 
plt.savefig(flag2 + '_lc_decay_34_R1e13_B8e-1_Doppler_' + flag + '.png',bbox_inches = 'tight')
plt.show()